In [15]:
import os, json
from pathlib import Path
import pandas as pd
import numpy as np
import glob
pd.set_option('display.max_columns', None)

In [16]:
path_to_json = Path('C:\\Users\\niti.mishra\\Documents\\Personal\\cyberbullying\\data\\labelled_tweets\\ab')
json_pattern = os.path.join(path_to_json,'*.json')
file_list = glob.glob(json_pattern)
# file_list = file_list[:-2] 
file_list

['C:\\Users\\niti.mishra\\Documents\\Personal\\cyberbullying\\data\\labelled_tweets\\ab\\random_tweets_1b.json',
 'C:\\Users\\niti.mishra\\Documents\\Personal\\cyberbullying\\data\\labelled_tweets\\ab\\random_tweets_2a.json',
 'C:\\Users\\niti.mishra\\Documents\\Personal\\cyberbullying\\data\\labelled_tweets\\ab\\random_tweets_3a.json',
 'C:\\Users\\niti.mishra\\Documents\\Personal\\cyberbullying\\data\\labelled_tweets\\ab\\random_tweets_4b.json',
 'C:\\Users\\niti.mishra\\Documents\\Personal\\cyberbullying\\data\\labelled_tweets\\ab\\random_tweets_5b.json',
 'C:\\Users\\niti.mishra\\Documents\\Personal\\cyberbullying\\data\\labelled_tweets\\ab\\random_tweets_6a.json',
 'C:\\Users\\niti.mishra\\Documents\\Personal\\cyberbullying\\data\\labelled_tweets\\ab\\random_tweets_7b.json',
 'C:\\Users\\niti.mishra\\Documents\\Personal\\cyberbullying\\data\\labelled_tweets\\ab\\random_tweets_8a.json']

In [17]:
data = []
for file in file_list:
    with open(file, encoding='utf-8-sig') as f:
        for line in f:
            data.append(json.loads(line)) 

In [18]:
temp = pd.DataFrame(data)
temp.head()

,id,full_tweet,bullying_trace,bullying_role,form_of_bullying,bullying_post_type
0,1168174144724160518,@heyyyitsbieha bullying and helping is two dif...,yes,bully,general,denial
1,1162980168232853504,@umineko_lover @mienumirai i mean it has some ...,no,NaN,NaN,NaN
2,1161905202527526912,my take on social issues like this is that eve...,yes,reporter,general,report
3,1162217363023880193,i want to watch a group of characters ranging ...,no,NaN,NaN,NaN
4,1168710966659272705,i’d never let a fucking fetus on twitter bully...,no,NaN,NaN,NaN


In [19]:
tweets = temp.copy()
tweets = tweets[tweets['bullying_trace'].notna()]
tweets = tweets[tweets['bullying_trace']!= 'remove']
print(tweets.shape)

testtweet = temp['full_tweet'][0]
testtweet

(7868, 6)


'@heyyyitsbieha bullying and helping is two different things,i didn’t force him,im not using the rock’s voice or the big show’s slam,i’m just asking him if he’s free and would like to help,but i’m agreeing with you that bullying is a trash behavior!✌🏻'

In [20]:
label_names = tweets.columns[2:]
y = tweets[label_names[0]]

freq = y.value_counts()           # count frequency of different classes in loan status
freq/sum(freq)*100  

no     60.739705
yes    39.260295
Name: bullying_trace, dtype: float64

In [21]:
import nltk
from nltk import pos_tag
from nltk.tokenize import TweetTokenizer
# from six import string_types

In [22]:
tt = TweetTokenizer()

testtweet=tt.tokenize(testtweet)
tweets['full_tweet'] = tweets['full_tweet'].apply(tt.tokenize)
tweets.head()

,id,full_tweet,bullying_trace,bullying_role,form_of_bullying,bullying_post_type
0,1168174144724160518,"[@heyyyitsbieha, bullying, and, helping, is, t...",yes,bully,general,denial
1,1162980168232853504,"[@umineko_lover, @mienumirai, i, mean, it, has...",no,NaN,NaN,NaN
2,1161905202527526912,"[my, take, on, social, issues, like, this, is,...",yes,reporter,general,report
3,1162217363023880193,"[i, want, to, watch, a, group, of, characters,...",no,NaN,NaN,NaN
4,1168710966659272705,"[i, ’, d, never, let, a, fucking, fetus, on, t...",no,NaN,NaN,NaN


In [23]:
testtweet = pos_tag(testtweet)
tweets['full_tweet'] = tweets['full_tweet'].apply(pos_tag)
tweets.head()

,id,full_tweet,bullying_trace,bullying_role,form_of_bullying,bullying_post_type
0,1168174144724160518,"[(@heyyyitsbieha, NN), (bullying, NN), (and, C...",yes,bully,general,denial
1,1162980168232853504,"[(@umineko_lover, IN), (@mienumirai, NNP), (i,...",no,NaN,NaN,NaN
2,1161905202527526912,"[(my, PRP$), (take, NN), (on, IN), (social, JJ...",yes,reporter,general,report
3,1162217363023880193,"[(i, NN), (want, VBP), (to, TO), (watch, VB), ...",no,NaN,NaN,NaN
4,1168710966659272705,"[(i, NN), (’, VBP), (d, NN), (never, RB), (let...",no,NaN,NaN,NaN


In [24]:
def modtweet(tagged_tweets):
    mod_tweets = []
    for tweet in tagged_tweets:
        mod_tweet=[]
        for (token, tag) in tweet:    
            if '@' in token:
                token = '@user'
            elif '#' in token:
                token = token[1:]
            elif ('http' or 'https') in token:
                token = 'UR'
            elif token == 'luv':
                token = 'love'
            else:
                pass
            mod_tweet.append((token,tag))
        mod_tweets.append(mod_tweet)
    return mod_tweets

testtweet=modtweet([testtweet])
tweets['full_tweet'] = modtweet(tweets['full_tweet'])
tweets.head()

,id,full_tweet,bullying_trace,bullying_role,form_of_bullying,bullying_post_type
0,1168174144724160518,"[(@user, NN), (bullying, NN), (and, CC), (help...",yes,bully,general,denial
1,1162980168232853504,"[(@user, IN), (@user, NNP), (i, JJ), (mean, VB...",no,NaN,NaN,NaN
2,1161905202527526912,"[(my, PRP$), (take, NN), (on, IN), (social, JJ...",yes,reporter,general,report
3,1162217363023880193,"[(i, NN), (want, VBP), (to, TO), (watch, VB), ...",no,NaN,NaN,NaN
4,1168710966659272705,"[(i, NN), (’, VBP), (d, NN), (never, RB), (let...",no,NaN,NaN,NaN


In [25]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
import unicodedata

In [26]:
def is_punct(token):
    return all(
            unicodedata.category(char).startswith('P') for char in token
        )

def lemmatize(token, pos_tag):
    lemmatizer = WordNetLemmatizer()
    tag = {
            'N': wn.NOUN,
            'V': wn.VERB,
            'R': wn.ADV,
            'J': wn.ADJ
        }.get(pos_tag[0], wn.NOUN)
    return lemmatizer.lemmatize(token, tag)

def normalize(tagged_tweet):
    
    lemm_tweet = [lemmatize(token, tag).lower()
                  for (token, tag) in tagged_tweet
                  if not is_punct(token) #and not self.is_stopword(token)
                 ]
        
    return lemm_tweet


In [27]:
testtweet = normalize(testtweet[0])
tweets['full_tweet'] = tweets['full_tweet'].apply(normalize)
tweets.head()

,id,full_tweet,bullying_trace,bullying_role,form_of_bullying,bullying_post_type
0,1168174144724160518,"[@user, bullying, and, help, be, two, differen...",yes,bully,general,denial
1,1162980168232853504,"[@user, @user, i, mean, it, have, some, really...",no,NaN,NaN,NaN
2,1161905202527526912,"[my, take, on, social, issue, like, this, be, ...",yes,reporter,general,report
3,1162217363023880193,"[i, want, to, watch, a, group, of, character, ...",no,NaN,NaN,NaN
4,1168710966659272705,"[i, d, never, let, a, fucking, fetus, on, twit...",no,NaN,NaN,NaN


In [83]:
tweets.to_json(Path(path_to_json.parent, 'all_tweets.json'), orient='records', lines=True)